<a href="https://colab.research.google.com/github/buganart/BUGAN/blob/master/custom_dataset_of_script_VAEGAN_voxelsize1_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Mount google drive.
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

# Check if we have linked the folder
from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    print(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

In [ ]:
#@markdown Install wandb and log in
%pip install wandb
output.clear()
import wandb
!wandb login
output.clear()
print("ok!")

In [ ]:
#@title Configure dataset
#@markdown - Leave empty if you want to start a new run
#@markdown - Set `"run_id"` if you want to resume a run (for example: `u9imsvva`)
#@markdown - The id of the current run is shown below in the cell with `wandb.init()` or you can find it in the wandb web UI.
resume_id = "" #@param {type:"string"}
#@markdown Enter project name (either `handtool-gan` or `tree-gan`)
project_name = "tree-gan" #@param ["tree-gan", "handtool-gan"]
#@markdown Enter dataset location.  
#@markdown - For example via the file browser on the left to locate and right click to copy the path.)
#@markdown - zipfile example: `/content/drive/My Drive/h/k/a.zip`
#@markdown - file folder example: `/content/drive/My Drive/h/k` 
data_location = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj_auto_generated/sessions/simplified/tree-session-2020-09-14_23-23-Friedrich_2-target-face-num-1000.zip" #@param {type:"string"}
#@markdown - choose rotation augmentation on-the-fly 
#@markdown (augmentation only support file folder in data_location)
data_augmentation = True    #@param {type:"boolean"}
aug_rotation_type = "random rotation"  #@param ["random rotation", "axis rotation"]
#@markdown - specify the rotation axis [x,y,z] (only for aug_rotation_type = "axis rotation")
rotation_axis_x = 0    #@param {type:"number"}
rotation_axis_y = 1    #@param {type:"number"}
rotation_axis_z = 0    #@param {type:"number"}

#@markdown - resolution of the voxelized array (shape resolution**3)
resolution = "32"    #@param [32, 64]

#@markdown Model
#@markdown - select which model to train
selected_model = "VAEGAN"    #@param ["VAEGAN", "GAN", "VAE"]

#@markdown WANDB log
#@markdown - how many epochs before logging images/3D objects
log_interval = 5    #@param {type:"integer"}
#@markdown - how many samples per log
log_num_samples = 3    #@param {type:"integer"}


colab_config = {
    "aug_rotation_type": aug_rotation_type,
    "data_augmentation": data_augmentation,
    "data_location": data_location,
    "resume_id": resume_id,
    "log_interval": log_interval,
    "log_num_samples": log_num_samples,
    "project_name": project_name,
    "resolution": resolution
}

for k, v in colab_config.items():
    print(f"=> {k:20}: {v}")


# To just train a model, no edits should be required in any cells below.

In [ ]:
import os
from pathlib import Path
# os.environ["WANDB_MODE"] = "dryrun"

%cd /content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/
if project_name == "tree-gan":
    %cd colab-treegan/
else:
    %cd colab-handtool/

dataset_path = Path(data_location)
run_path = "./"

if data_location.endswith(".zip"):
    dataset_name = dataset_path.stem
else:
    dataset_name = "dataset_array_custom"

!apt-get update

%pip install pytorch-lightning
%pip install trimesh
!apt install -y xvfb
%pip install trimesh xvfbwrapper
output.clear()
print('ok!')

In [ ]:
import io
from io import BytesIO
import zipfile
import trimesh
import numpy as np
from argparse import Namespace


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader, TensorDataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

from xvfbwrapper import Xvfb

In [ ]:
if not resume_id:
    run_id = wandb.util.generate_id()
    resume = False
else:
    run_id = resume_id
    resume = True

run = wandb.init(project=project_name, id=run_id, entity="bugan", resume=True, dir=run_path, group=selected_model)

print("run id: " + str(wandb.run.id))
print("run name: " + str(wandb.run.name))
wandb.watch_called = False

In [ ]:
#keep track of hyperparams
config = Namespace(**colab_config)

config.epochs = 3000
config.batch_size = 32
config.array_size = int(resolution)

#model param
config.z_size = 128
if config.array_size == 32:
    config.gen_num_layer_unit = [1024, 512, 256, 128]
    config.dis_num_layer_unit = [32, 64, 128, 128]
else:
    config.gen_num_layer_unit = [1024, 512, 256, 128, 128]
    config.dis_num_layer_unit = [32, 64, 64, 128, 128]

if selected_model == "VAEGAN":
    config.vae_recon_loss_factor = 1
    config.vae_opt = "Adam"
    config.dis_opt = "Adam"
    config.vae_lr = 0.0025
    config.vae_encoder_layer = 1
    config.vae_decoder_layer = 2
    config.d_lr = 0.00005            
    config.d_layer = 1
elif selected_model == "GAN":
    config.gen_opt = "Adam"
    config.dis_opt = "Adam"
    config.g_lr = 0.0025
    config.g_layer = 2
    config.d_lr = 0.00005           
    config.d_layer = 1
else:
    config.vae_opt = "Adam"
    config.vae_lr = 0.0025
    config.vae_encoder_layer = 1
    config.vae_decoder_layer = 2

config.seed = 1234
config.aug_rotation_axis = (rotation_axis_x,rotation_axis_y,rotation_axis_z)



#dataset

In [ ]:
### load our package

#directly install using pip
%pip install git+https://github.com/buganart/BUGAN.git#egg=bugan
output.clear()

from bugan.functionsPL import *
from bugan.modelsPL import VAEGAN, VAE_train, GAN, VAE, Discriminator, Generator


run.tags.append(selected_model)

###     load dataset
np.random.seed(config.seed)

config.dataset = dataset_name
dataModule = DataModule_process(config, run, dataset_path)
config.num_data = dataModule.size

#train

In [ ]:
#set seed
torch.manual_seed(config.seed)
torch.autograd.set_detect_anomaly(True)

#render setup
vdisplay = Xvfb()
vdisplay.start()

#wandb logger setup
wandb_logger = WandbLogger(experiment=run, log_model=True)
#log config
wandb.config.update(config)

checkpoint_path = str(Path(wandb.run.dir).absolute()/'checkpoint.ckpt')
callbacks = [SaveWandbCallback(config.log_interval, checkpoint_path)]


if resume:
    # Download file from the wandb cloud.
    load_checkpoint_from_cloud(checkpoint_path = 'checkpoint.ckpt')
    extra_trainer_args = {'resume_from_checkpoint': checkpoint_path}
else:
    extra_trainer_args = {}

trainer = pl.Trainer(max_epochs = config.epochs, logger=wandb_logger, callbacks=callbacks, checkpoint_callback = None, **extra_trainer_args)

#model
if selected_model == "VAEGAN":
    model = VAEGAN(config).to(device)
elif selected_model == "GAN":
    model = GAN(config).to(device)
else:
    model = VAE_train(config).to(device)
wandb_logger.watch(model)


#train
trainer.fit(model, dataModule)